### 题目描述

我们提供了一个类：

```
public class Foo {
  public void one() { print("one"); }
  public void two() { print("two"); }
  public void three() { print("three"); }
}
```

三个不同的线程将会共用一个 Foo 实例。请设计修改程序，以确保 two() 方法在 one() 方法之后被执行，three() 方法在 two() 方法之后被执行。

### 题目分析

题目要求，无论三个线程启动的顺序如何，但其最终的输出要保证特定的顺序。这是并发问题。

并发计算场景下，程序在多线程（或多进程）中同时执行。这里的同时执行，不是指不同的线程或者进程在不同的物理CPU上分别运行，更多的是在同一个CPU上交替执行多个线程或进程。此时会出现几个问题：

- 竞态条件：由于多进程之间的竞争执行，导致程序未按照期望的顺序输出。
- 死锁：并发程序等待一些必要资源，导致没有程序可以执行。
- 资源不足：进程被永久剥夺了运行所需的资源。

此题目中存在竞态条件。并发问题中，多个线程或进程共享某些资源，由于无法消除资源共享的约束，防止并发问题就变成了资源共享的协调问题。如果可以保证程序中关键部分代码的独占性，就可以防止程序进入不一致的状态。

竞争条件的解决方案为： 需要某些关键部分代码具有排他性，即在给定的时间内，只有一个线程可以进入关键部分代码。

题目要求按顺序依次执行三个方法，且每个方法都在单独的线程中运行。为了保证线程的执行顺序，可以在方法之间创建一些依赖关系，即第二个方法必须在第一个方法执行完之后开始执行，第三个方法必须在第二个方法执行完之后执行。

In [1]:
from threading import Lock
class Foo:
    def __init__(self):
        self.firstJobDone = Lock()
        self.secondJobDone = Lock()
        self.firstJobDone.acquire()
        self.secondJobDone.acquire()


    def first(self, printFirst: 'Callable[[], None]') -> None:
        
        # printFirst() outputs "first". Do not change or remove this line.
        printFirst()
        self.firstJobDone.release()


    def second(self, printSecond: 'Callable[[], None]') -> None:
        
        # printSecond() outputs "second". Do not change or remove this line.
        with self.firstJobDone:
            printSecond()
            self.secondJobDone.release()


    def third(self, printThird: 'Callable[[], None]') -> None:
        
        # printThird() outputs "third". Do not change or remove this line.
        with self.secondJobDone:
            printThird()